#### Se instalan las librerias principales

In [ ]:
!pip install requests
!pip install fastparquet

#### Se importan las Librerias

In [63]:
import requests
import pandas as pd
import os

#### Seccion funciones que extraen de las API

In [7]:
# Esta funcion extrae las coordenadas geograficas de la ciudad ingresada.
# Devuelve un df con los resultados.

def get_city(cityName, country, headers):
    baseUrl = 'https://api.api-ninjas.com/v1/geocoding?city=' + cityName + '&country=' + country
    response = requests.get(baseUrl, headers)
    adress = response.json()
    if len(adress) == 0:
        print('Ciudad no encontrada')
        return pd.DataFrame()
    else:
        return pd.DataFrame.from_dict(adress)

       
def get_weather(cityCoord, fechaStart, fechafin):
    clima = 'https://api.open-meteo.com/v1/forecast?latitude=' + str(cityCoord.iloc[0,1]) + '&longitude=' + str(cityCoord.iloc[0,2]) + '&hourly=temperature_2m,wind_speed_10m&start_date=' + fechaStart + '&end_date=' + fechafin
    
    response = requests.get(clima)
    response_dict = response.json()
    return pd.DataFrame.from_dict(response_dict['hourly'])
    

#### Seccion funciones de verificacion de datos

In [5]:
# Esta funcion intentria verificar si se encuentra mas de una ciudad con el mismo nombre en un mismo pais.
# La dejo para mas adelante que el usuario por ejemplo seleccione el numero de la ciudad deseada.

def check_city(cityCoordDf):
    if len(cityCoordDf) == 0:
        print('Por favor ingrese correctamente el nombre de la ciudad')
    elif len(cityCoordDf) == 1:
        print('Ciudad encontrada correctamente')
        print(cityCoordDf.iloc[0,0])
        return cityCoordDf
    else:
        print(cityCoordDf)
        num = int(input("Elija el numero de la ciudad deseada: "))
        #selectedCityCode = cityCoordDf.iloc[num,]
        #check_city(selectedCityCode)

#### Seccion Resguardo de Datos

In [67]:
def saveDataCity(df):
    
    directoryPath = 'weather/' + df.iloc[0,0] + '/'
    dfName = directoryPath + df.iloc[0,0] + '.csv'
    
    directory = os.path.dirname(directoryPath)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
    
    df.to_csv(dfName, sep='\t')
       

def saveDataParquet(df, outputPath, partitionCols=None):
    
    directory = os.path.dirname(outputPath)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
    
    df.to_parquet(
        outputPath,
        engine = 'fastparquet',
        partitionCols = partitionCols)
    
    

#### Ingreso de datos para extraccion

In [8]:
# Ingreso la ciudad a buscar.
cityName = 'pico truncado'
country = 'argentina'
headers = {'X-Api-Key': 'YO3DOw3APQguBKsqQq0nMA==acw96vJrBQsSnpeG'}

cityCoordDf = get_city(cityName, country, headers)

cityCoord = check_city(cityCoordDf)

NameError: name 'requests' is not defined

In [70]:
# Como la ciudad se extrajo con exito
# Ahora consulto el clima

# Antes se consulta las fechas a extraer.
fechaStart = '2024-03-09'
fechafin = '2024-03-10'

weather = get_weather(cityCoord, fechaStart, fechafin)

weather.head

<bound method NDFrame.head of                 time  temperature_2m  wind_speed_10m
0   2024-03-09T00:00            29.5             8.9
1   2024-03-09T01:00            28.1             6.2
2   2024-03-09T02:00            27.2             3.4
3   2024-03-09T03:00            26.5             3.7
4   2024-03-09T04:00            26.0             1.6
5   2024-03-09T05:00            25.7             1.5
6   2024-03-09T06:00            24.8             3.3
7   2024-03-09T07:00            23.8             4.7
8   2024-03-09T08:00            23.7             4.7
9   2024-03-09T09:00            23.7             5.1
10  2024-03-09T10:00            23.6             5.1
11  2024-03-09T11:00            23.4             5.0
12  2024-03-09T12:00            25.1             5.9
13  2024-03-09T13:00            27.4             4.7
14  2024-03-09T14:00            29.5             2.9
15  2024-03-09T15:00            31.0             6.0
16  2024-03-09T16:00            32.3             9.8
17  2024-03-09T1

In [ ]:
#Genero el directorio para esa ciudad
saveDataCity(cityCoord)